In [51]:
import numpy as np
import csv
import matplotlib.pyplot as plt

In [52]:
path_to_mtrx = '/home/gabridele/Desktop/connectome_sub-100610.csv'
#path_to_mtrx = '/Users/gabrieledele/Desktop/Hub_identification/gabriele_sc_connectomes/connectome_sub-100610.csv'
original_matrix = np.loadtxt(path_to_mtrx, delimiter=',')
resized_matrix = original_matrix[:100, :100]
print(resized_matrix.shape)

(100, 100)


In [132]:
import numpy as np
import matplotlib.pyplot as plt

# Load the connectivity matrix from a CSV file
def load_connectivity_matrix(file_path):
    return np.loadtxt(file_path, delimiter=',')

# Set the activation threshold
## che non ci siano righe/cols con zero
## to each subj their threshold
## far andare simulazione

# Select seed nodes randomly
def select_seed_nodes(num_nodes, num_seeds):
    return np.random.choice(num_nodes, num_seeds, replace=False)

def find_highest_threshold(connectivity_matrix, threshold_values, max_iterations=10):
    weighted_degree = np.sum(connectivity_matrix, axis=0)
    lowest_weighted_degree = np.min(weighted_degree)
    print("Lowest weighted degree:", lowest_weighted_degree)
    index_lowest_degree = np.argmin(weighted_degree)
    print("with index:", index_lowest_degree)
    seed_nodes = index_lowest_degree

    successful_thresholds = []  # List to store successful threshold values
    for threshold in threshold_values:
        breakcycle = False
        print('threshold:', threshold)
        num_nodes = connectivity_matrix.shape[0]
        active_nodes = np.zeros(num_nodes, dtype=bool)
        active_nodes[seed_nodes] = True

        iteration = 0  # Initialize iteration counter
        
        while not np.all(active_nodes) and iteration < max_iterations:
            new_active_nodes = np.zeros(num_nodes, dtype=bool)
            for node_idx in range(num_nodes):
                if not active_nodes[node_idx]:
                    neighbors = connectivity_matrix[node_idx] 
                    neighbors_activated = active_nodes[np.where(neighbors > 0)[0]]
                    degree_of_node = np.sum(connectivity_matrix[node_idx] > 0)
                    activation_condition = threshold * degree_of_node < np.sum(neighbors_activated)
                    if activation_condition:
                        new_active_nodes[node_idx] = True        
            active_nodes = np.logical_or(active_nodes, new_active_nodes)

            iteration += 1
            print(f"Iteration {iteration}: Number of active nodes = {np.sum(active_nodes)}")

        if iteration == max_iterations:
            breakcycle = True
            print("Maximum number of iterations reached.")
        else:
            print("Simulation complete.")
        if breakcycle == True:
            break
        else:
            if np.all(active_nodes):
                print(f"Successful simulation with threshold: {threshold}")
                successful_thresholds.append(threshold)
    
    if successful_thresholds:
        highest_threshold = max(successful_thresholds)
        print("Highest threshold allowing simulation completion:", highest_threshold)
        return highest_threshold
    else:
        print("Unable to find a threshold allowing simulation completion.")
        return None
    
def simulate_activation(connectivity_matrix, seed_nodes, activation_threshold, max_iterations):
    num_nodes = connectivity_matrix.shape[0]
    print('num_nodes:', num_nodes)
    print('seed_nodes:', seed_nodes)
    active_nodes = np.zeros(num_nodes, dtype=bool)
    print('active_nodes:', active_nodes) 
    # Everything should be false, because created a new array with same num nodes and zeroed out, except for the seeds
    active_nodes[seed_nodes] = True
    print('active_nodes2:', active_nodes[seed_nodes])
    print('active_nodes3:', active_nodes)
    
    iteration = 0  # Initialize iteration counter
    
    while not np.all(active_nodes) and iteration < max_iterations: # Checks if there are still inactive nodes. Continues till all nodes are activated
        new_active_nodes = np.zeros(num_nodes, dtype=bool) # Array to store nodes that become activated in current iteration
        # Initially all nodes are inactive
        for node_idx in range(num_nodes): # Iterates over each node in the network
            if not active_nodes[node_idx]: #  Checks if current node is inactive
                neighbors = connectivity_matrix[node_idx] # Retrieves row corresponding to the current node, aka neighbors
                # Selects activated neighbors of current node by filtering active_nodes array based on non-zero entries in neighbors row
                neighbors_activated = active_nodes[np.where(neighbors > 0)[0]]
                degree_of_node = np.sum(connectivity_matrix[node_idx] > 0)  # Degree of the node by counting non-zero entries in row
                # Check if the number of activated neighbors exceeds the threshold
                # Threshold should be highest values that allows completion of cascade
                activation_condition = activation_threshold * degree_of_node < np.sum(neighbors_activated)
                if activation_condition:
                    # If so, then node becomes activated
                    new_active_nodes[node_idx] = True
        # Updates active_nodes array by performing a logical OR operation with new_active_nodes array 
        # Activated nodes from the current iteration are added to the overall set of activated nodes          
        active_nodes = np.logical_or(active_nodes, new_active_nodes)

        
        iteration += 1
        print(f"Iteration {iteration}: Number of active nodes = {np.sum(active_nodes)}")
    
    print("Simulation complete.")
    return active_nodes

# Main function
def main():
    # Load the connectivity matrix
    connectivity_matrix = original_matrix
    
    # Select seed nodes
    num_nodes = connectivity_matrix.shape[0]
    num_seeds = 2

    # Find the highest threshold allowing simulation completion
    threshold_values = np.arange(0, 0.01, 0.0001).tolist()  # Generate threshold values
    highest_threshold = find_highest_threshold(connectivity_matrix, threshold_values)
    
    for _ in range(100):
        seed_nodes = select_seed_nodes(num_nodes, num_seeds)
        active_nodes = simulate_activation(connectivity_matrix, seed_nodes, highest_threshold, max_iterations=100)
        print("Final activation state with highest threshold:", active_nodes)
    print('For loop complete')

    
    ### Adjust later
    # Plot the results (Iterations vs. Number of Active Nodes)
    #plt.figure(figsize=(8, 6))
    #plt.plot(range(len(activated_nodes_counts)), activated_nodes_counts, marker='o', color='red')
    #plt.xlabel('Iterations')
    #plt.ylabel('Number of Active Nodes')
    #plt.title('Progress of Activation')
    #plt.grid(True)
    #plt.show()
    

if __name__ == "__main__":
    main()

Lowest weighted degree: 0.13396973500000003
with index: 211
threshold: 0.0
Iteration 1: Number of active nodes = 54
Iteration 2: Number of active nodes = 1029
Iteration 3: Number of active nodes = 1054
Simulation complete.
Successful simulation with threshold: 0.0
threshold: 0.0001
Iteration 1: Number of active nodes = 54
Iteration 2: Number of active nodes = 1029
Iteration 3: Number of active nodes = 1054
Simulation complete.
Successful simulation with threshold: 0.0001
threshold: 0.0002
Iteration 1: Number of active nodes = 54
Iteration 2: Number of active nodes = 1029
Iteration 3: Number of active nodes = 1054
Simulation complete.
Successful simulation with threshold: 0.0002
threshold: 0.00030000000000000003
Iteration 1: Number of active nodes = 54
Iteration 2: Number of active nodes = 1029
Iteration 3: Number of active nodes = 1054
Simulation complete.
Successful simulation with threshold: 0.00030000000000000003
threshold: 0.0004
Iteration 1: Number of active nodes = 54
Iteration 2